In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from geojson import GeoJSON
import json
import matplotlib
from numpy import mean, all, median

from bokeh.models import (ColumnDataSource, GMapPlot, Range1d, ColumnDataSource, LinearAxis,
HoverTool, PanTool, WheelZoomTool, BoxZoomTool, ResetTool,CustomJS,
GMapOptions, NumeralTickFormatter, PrintfTickFormatter, MapOptions)
from bokeh.models.glyphs import Patches
from bokeh.plotting import figure, save, show
from bokeh.resources import CDN
from bokeh.io import output_file, show, saving
from bokeh.models.widgets import RadioButtonGroup
from bokeh.layouts import widgetbox, column
from bokeh.models.widgets import Panel, Tabs
outfp = r"./map_display.html"
from bokeh import tile_providers
from bokeh.tile_providers import CARTODBPOSITRON

df_wash = pd.read_pickle('Washington.pickle')
df_wash.head()

df = df_wash[df_wash['total_score'].apply(len)>0].copy()
# This will discard all the hexes where there aren't any observations.
# remember to make this a copy or else you will end up with warnings about writing to a slice
df.head()

,grid,x_shape,y_shape,total_score,bike_score,car_score,mass_score,ride_score,points_list
id,,,,,,,,,
121,"(0, 0, 0)","[-8574909.759447943, -8575462.629447944, -8576...","[4708069.611732003, 4709027.210661984, 4709027...","[77, 83, 62, 93, 69, 56, 89, 97, 62, 65, 80, 6...","[0.0, 18.0, 0.0, 20.0, 23.0, 0.0, 22.0, 11.0, ...","[0.0, 4.0, 24.0, 4.0, 18.0, 14.0, 21.0, 3.0, 0...","[74.0, 57.0, 26.0, 61.0, 23.0, 18.0, 40.0, 73....","[26.0, 21.0, 50.0, 15.0, 36.0, 68.0, 17.0, 13....","[243225, 284325, 294461, 235844, 220630, 22479..."
121,"(10, -1, -9)","[-8558323.659447944, -8558876.529447945, -8559...","[4715730.4031718485, 4716688.002101829, 471668...","[77, 83, 62, 93, 69, 56, 89, 97, 62, 65, 80, 6...","[0.0, 18.0, 0.0, 20.0, 23.0, 0.0, 22.0, 11.0, ...","[0.0, 4.0, 24.0, 4.0, 18.0, 14.0, 21.0, 3.0, 0...","[74.0, 57.0, 26.0, 61.0, 23.0, 18.0, 40.0, 73....","[26.0, 21.0, 50.0, 15.0, 36.0, 68.0, 17.0, 13....","[243225, 284325, 294461, 235844, 220630, 22479..."
121,"(-10, 1, 9)","[-8591495.859447943, -8592048.729447944, -8593...","[4700408.820292158, 4701366.419222139, 4701366...","[77, 83, 62, 93, 69, 56, 89, 97, 62, 65, 80, 6...","[0.0, 18.0, 0.0, 20.0, 23.0, 0.0, 22.0, 11.0, ...","[0.0, 4.0, 24.0, 4.0, 18.0, 14.0, 21.0, 3.0, 0...","[74.0, 57.0, 26.0, 61.0, 23.0, 18.0, 40.0, 73....","[26.0, 21.0, 50.0, 15.0, 36.0, 68.0, 17.0, 13....","[243225, 284325, 294461, 235844, 220630, 22479..."


In [2]:
colors = [
'#ffffcc',
'#d9f0a3',
'#addd8e',
'#78c679',
'#31a354',
'#006837'
]
descriptions = ['Bad','Poor','Average','Good','Great','Excellent']
bins = [0,50,60,70,80,90,100]

def list_cruncher(df, col, function, c_list, d_list):
    string0 = function.__name__+ "_value_"+ col
    string1 = function.__name__+ "_"+ col
    string2 = string1 + "_ratings"
    temp = df[col].apply(lambda x: function(x)).copy()
    
    
    df[string0] = temp
    df[string1] = pd.cut(temp, bins, labels=c_list)
    df[string2] = pd.cut(temp, bins, labels=d_list)

list_cruncher(df,'total_score', mean, colors, descriptions)
list_cruncher(df,'total_score', median, colors, descriptions)
df.head(2)

,grid,x_shape,y_shape,total_score,bike_score,car_score,mass_score,ride_score,points_list,mean_value_total_score,mean_total_score,mean_total_score_ratings,median_value_total_score,median_total_score,median_total_score_ratings
id,,,,,,,,,,,,,,,
121,"(0, 0, 0)","[-8574909.759447943, -8575462.629447944, -8576...","[4708069.611732003, 4709027.210661984, 4709027...","[77, 83, 62, 93, 69, 56, 89, 97, 62, 65, 80, 6...","[0.0, 18.0, 0.0, 20.0, 23.0, 0.0, 22.0, 11.0, ...","[0.0, 4.0, 24.0, 4.0, 18.0, 14.0, 21.0, 3.0, 0...","[74.0, 57.0, 26.0, 61.0, 23.0, 18.0, 40.0, 73....","[26.0, 21.0, 50.0, 15.0, 36.0, 68.0, 17.0, 13....","[243225, 284325, 294461, 235844, 220630, 22479...",74.912383,#78c679,Good,76.0,#78c679,Good
121,"(10, -1, -9)","[-8558323.659447944, -8558876.529447945, -8559...","[4715730.4031718485, 4716688.002101829, 471668...","[77, 83, 62, 93, 69, 56, 89, 97, 62, 65, 80, 6...","[0.0, 18.0, 0.0, 20.0, 23.0, 0.0, 22.0, 11.0, ...","[0.0, 4.0, 24.0, 4.0, 18.0, 14.0, 21.0, 3.0, 0...","[74.0, 57.0, 26.0, 61.0, 23.0, 18.0, 40.0, 73....","[26.0, 21.0, 50.0, 15.0, 36.0, 68.0, 17.0, 13....","[243225, 284325, 294461, 235844, 220630, 22479...",74.912383,#78c679,Good,76.0,#78c679,Good


In [3]:


output_file(outfp)
dsource = ColumnDataSource(df)

class plot_pipeline:
    def __init__(self, source, center):
        self.source = source
        self.make_fig(center[0],center[1])
    
    def make_fig(self, x, y):
        tips = [
                ('Total Score', "@mean_value_total_score"),
                ('Description', "@mean_total_score_ratings")
        ]
        wheel = WheelZoomTool()
        tool_box = [HoverTool(tooltips=tips), wheel, PanTool(),
                 BoxZoomTool(), ResetTool()]
        bound = 15000 # 15 km
        plot = figure(x_range=Range1d(x-bound,x+bound),
            y_range=Range1d(y-bound, y+bound),
            toolbar_location="right",
            tools = tool_box, plot_width=1100,
            plot_height=1100)
        
        plot.toolbar.active_scroll = wheel
        plot.add_tile(CARTODBPOSITRON)
        self.plot=plot
        
    def add_layer(self, column_name):       
        
        hexes = Patches(xs='x_shape', ys='y_shape',
            fill_color=column_name, fill_alpha=0.4, 
            line_color="black", line_width=1.00)
        
        self.plot.add_glyph(dsource, hexes)
        
    def out(self):
        show(self.plot)
        
        
import pyproj
wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
webdb=pyproj.Proj("+init=EPSG:3857") # Web Mercator

washington_y = 38.904978
washington_x = -77.039658

x, y = pyproj.transform(wgs84, webdb, washington_x, washington_y)

        
wash = plot_pipeline(dsource, (x,y))
wash.add_layer('mean_total_score')
wash.out()



In [69]:
# wheel = WheelZoomTool()

# tool_box = [wheel, PanTool(),HoverTool(),
#          BoxZoomTool(), ResetTool()]


# bound = 15000 
# fig = figure() , tools=tool_box)
# fig.axis.visible = False
# fig
# output_file("wash_plot_maps.html")
# show(fig)